# Multi Voxel Pattern Analysis MVPA 
_______________________________________________________

# Table of Contents

1. [Overview](#overview)
2. [MVPA using Princeton Toolbox](#princeton)
3. [MVPA using Searchmigh Toolbox](#searchmight)

## Overview <a id="overview"></a>


MVPA allows to detect differences between conditions with higher sensitivity than conventional univariate analysis by focusing on the analysis and comparison of distributed patterns of activity, under this approach the data from individual voxels within a region are jointly analyzed. Thus MVPA is often presented in the context of _brain reading_ applications intending to report that specific **mental states** or **representational content** can be decoded from fMRI activity patterns after performing a _training_ or _learning phase_. MVPA tools are referred as classifiers or learning machines, below there is a list of some of the tools used within different softwares in order to perform this analysis.

1. Multivariate _searchlight mapping_ approach (Kriegeskorte, Goebel & Bandettini, 2006)
2. ROI-Support Vector Machine (SVM), is used both for analyzing patterns in ROIs and for discriminating patterns that are potentially spread out across the whole brain.
3. Recursive Feauture Elimination (RFE) is a technique used to restrict the number of voxels (de Martino et al. 2008)

Before performing an MVPA it is necessary to think carefully about what types of activity changes are likely to be detected in your fMRI activation data. In case one might want to detect a difference in the amount of BOLD (v.g. there will be more BOLD in a certain brain region during condition A in comparison to condition B), a better approach is to perform a mass-univariate/GLM/SPM analysis. Alternatively one might want to detect consistent differences regardless of the amount or direction of BOLD change (v.g. the BOLD for a certain brain region will be different between a set of conditions), then MVPA is appropiate.

The ultimate goal is to find a model capable of identify the correspondence of an activation pattern with stimulus or events of certain category.

For other insights on how to begin to perform an MVPA and a list of related literature follow this [link](http://mvpa.blogspot.com/2013/02/where-to-start-with-mvpa.html)

Brainvoyager offers a comprehensive exposition of this topic by following this [link](http://www.brainvoyager.com/bvqx/doc/UsersGuide/MVPA/BasicConcepts.html)

**References**

De Martino F, Valente G, Staeren N, Ashburner J, Goebel R., Formisano E. (2008). Combining multivariate voxel selection and support vector machines for mapping and classification of fMRI spatial patterns. NeuroImage, 43, 44-48.

Kriegeskorte N, Goebel R, Bandettini P (2006). Information-based functional brain mapping. PNAS, 103, 3863-3868.

## MVPA using Princeton Toolbox  <a id="princeton"></a>


This is a toolbox that uses the MATLAB environment in order to perform multi-voxel pattern analysis of fMRI neuroimaging data. The toolbox is availaible in a Github repository. [link to the repo](https://github.com/princetonuniversity/princeton-mvpa-toolbox). 

### Setting up MATLAB to perform MVPA ###

In order to use _Princeton MVPA toolbox_ you are required to work on MATLAB and have previously installed _Neuroelf_ in its 0.9c version. After you have downloaded the toolbox set the path to the toolbox file, additionally and in order to perform an MVPA download the AFNI data set from Haxby et al and put it in the same path. Verify that you have correctly set up your working environment by running the script below into MATLAB's command window.

In [ ]:
# Installing Neuroelf
NeuroElf_v09c -i
neuroelf_setup
neuroelf_version
# Adding the path of the toolbox e.g.
addpath /Users/williamcruz/Documents/MATLAB_2016/princeton_mvpa_toolbox
savepath

help tutorial_easy

In the study of Haxby et al participants viewed a lot of exemplars from different classes of visual objects (i.e. face, house, cat, bottle, scissors, shoes, chair, scramble). Each run contained a 9-TR block for each of the conditions mentioned above with rest in between and either end, the _working data set_ contains 10 runs acquired from a single subject. Given this data set the goal of applying MVPA is to being able to clasify distributed patterns of a single subject's neural activity in order to predict his or her psychological representation from the neural activity correlate. The required information in order to proceed with the analysis is as follows

1. The sample data set in AFNI format (BRIK/HEAD files) from Haxby et al.: This corresponds neural activation given different conditions to be classified.
2. Regressors Information - _tutorial_regs.mat_: This corresponds to a matrix of labels for the conditions (i.e. The categories in the study) that identify each volume in the data, such information is necessary for training the classifier.
3. Runs information - _tutorial_runs.mat_: This file contains information about what time points can be taken for the training and for the test stages, thus it works as a _selector_.
4. A mask in AFNI format: The tutorial don't use a whole brain approximation so in this case the mask contains a boolean 3D spatial labeling the voxels we want to either keep or exclude.

### Creation of a _subj_ structure

We begin by creating a _subj_ structure, this is done by loading the previously mentioned files: activation patterns, regressors, selectors and the mask. This subject structure will be used by the classifier and its supporting functions. The next step involves creating a _mask_ using the `load_afni_mask` function to restrict the voxels we want to allow and add it to the _subj_ structure. Following, we create a pattern object called _epi_ using `init_object` to store the raw EPI voxel data which will restrict the voxels to be included in the analysis. The next step consists in loading the regressor matrix to the _subj_ structure, this matrix stores information about the **TR** conditions, with similar properties as the regressors matrix of a GLM. These regressors are used for training the classifier and assessing whether the classification trial was successful or not. A similar process needs to be performed for the field _selector_, this matrix stores information about which of the 10 runs each TR came from in the form of vector containing time labels. The following perform the steps above: the creation of the _subj_ structure and for loading all of the required information in.

**Note**: For converting the _nifti_ format files (n) to brick/head use the AFNI command `3dcopy`
````
    3dcopy myfile.nii myfile
````
will create `myfile+tlrc.HEAD` and `myfile+tlrc.BRIK`.Alternatively
````
    mv gefunct1+orig.nii gefunct1.nii
    3dcopy gefunct1.nii gefunct1
    
    3dcopy -help # for mor information
````

In [ ]:
% Loading Mask
subj = init_subj('haxby8','tutorial_subj');
subj = load_afni_mask(subj,'VT category-selective','mask_cat_select_vt+orig');
% Loading sample data
for i=1:10
  raw_filenames{i} = sprintf('haxby8_r%i+orig',i);
end
subj = load_afni_pattern(subj,'epi','VT category-selective',raw_filenames);
% Loading conditions regressors
subj = init_object(subj,'regressors','conds');
load('tutorial_regs'); % 'tutorial_regs.mat'
subj = set_mat(subj,'regressors','conds',regs);
condnames = {'face','house','cat','bottle','scissors','shoe','chair','scramble'};
subj = set_objfield(subj,'regressors','conds','condnames',condnames);
% Loading runs selector
subj = init_object(subj,'selector','runs');
load('tutorial_runs'); % 'tutorial_runs.mat'
subj = set_mat(subj,'selector','runs',runs);

### Pre-classification steps

Once we have loaded everything we are ready to start the pre-classification steps; so when using the structure as the input into the `summarize` function, we should be able to see the properties of the _subj_ structures, indicating that we are ready to transform and slim this data before passing it into the classifier.

In [ ]:
summarize(subj)

Before performing the classification procedure it is helpful to _zscore_ the data by subtracting out the mean of each voxel's time course and scaling it in order to set the standard deviation of the time course to be 1. The previous is done by using the `zscore_runs` function indicating both, the runs and the selector name information, this z-score the EPI data individually on each run. The next step consist in creatint the group of selector indices for the different iterations of the n-minus-one cross-validation training scheme. Following, we are going to rule out the uninformative voxels by performing ANOVAs as method for _feature selection_. The ANOVA results will indicate the probability that a given voxel's activity varies significantly between conditions. The above can be performed by using the `feature_select` function, which will store the p-values from the ANOVA as a pattern.m

In [ ]:
% z-scoring
subj = zscore_runs(subj,'epi','runs');
% Creating the selector indices
subj = create_xvalid_indices(subj,'runs');
% Running ANOVA iterations
[subj] = feature_select(subj,'epi_z','conds','runs_xval');

### Cross-validation classification

Finally and for performing the cross-validation classification we will set some arguments for a backprop classifier with no hidden layer. First, we call the `cross_validation` function to classify the _epi_z_ data according to the _conds_ (conditions), which will call backprop for each iteration, iterating through the _runs_xval_ selector and _epi_z_thresh0.05_ mask previously created. Now the generated results can be invoked by typing `results` in the MATLAB command window. The _total_perf_ indicates the overall proportion of times that the classifier guessed correctly when testing TRs averaged across the iterations. In this case the performance value for the classifier was 0.4942, in other words it accurately guessed the condition to which it belongs 49% of the times, which in comparison to the 12.5% that we will get from a random classifier is not bad.

In [ ]:
% Set the arguments for a backprop classifier
class_args.train_funct_name = 'train_bp';
class_args.test_funct_name = 'test_bp';
class_args.nHidden = 0;
% Running classification, training and test
[subj results] = cross_validation(subj,'epi_z', 'conds', 'runs_xval','epi_z_thresh0.05',class_args);
% Performance of the classification algorithm
results

## MVPA using _searchmight_ toolbox <a id="searchmight"></a>

Searchmight is a toolbox designed for _searchlight maps_ using machine learning classifiers, aiming to make comparison of several different classifiers feasible, and provide a unified statistical testing procedure for the resulting maps. To know more about the toolbox's features visit the online [toolbox repository](http://www.franciscopereira.org/searchmight/).
In order to use the _searchmight_ toolbox and perform the MVPA using the following pipeline you are required to work on MATLAB and have previously installed _Neuroelf_ in its 1.1 version. For completing this pipeline is recommended to have both toolboxes _searchmight_ as well as _princeton_ MVPA in the same path along with some of the previously created files from the CG OBJECTS tutorial data set (i.e. _.sdm_, _.vtc_, _.vmp_). Additionally we used as the mask file (v.g. in the format .msk) the one from the automated meta-analysis on the **Visual Cortex: association test** available in the neurosynth.org [website](http://neurosynth.org); as in the previous example, the mask will restrict the voxels we want to allow include in the analyses so we found pertinent to use the one provided in the meta-analysis on _Neurosynth_.

In the CG OBJECTS tutorial data set the subject view either images in the Right Visual Field (RVF), images in the Left Visual Field (LVF) or images in Both Visual Field (BVF). Each single run contained a 3-TR block for each of the conditions mentioned above with Fixations in between and at either end. For facilitating the implementation of a MVPA the BVF condition was excluded, so in total we have 6 blocks/epochs for training/testing the classifier. After following the lines of MALTAB script below we obtained that our classifier had an accuracy performance of 0.83%, indicating a good performance value. Finally and as indicated below the cross-validation implemented a library for Support Vector Machines (i.e. LIBSVM, which can be downloaded in the following [link](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)) algorithm, so make sure that you have the latest version of it within the _CoreToolbox_ folder of the _Searchmight_ toolbox. When using the _LIBSVM_ classification algorithm one of the blocks is used for testing in each iteration.

In [ ]:
neuroelf_setup
subj_list = {'cg'};
% Loading and Creating Mask
maskfile = 'visual_field_neurosynth.msk';
[pathstr, mask_name, ext] = fileparts(maskfile);
maskObj = xff(sprintf('%s', mask_name, '.msk')); # make sure you are in the location 
mask = maskObj.Mask;
meta = createMetaFromMask(mask);
nVoxels = length(meta.indicesIn3D);
subj= init_subj('CG_Objects','CG');
subj = load_bv_mask(subj,mask_name,mask);
vmp_list = [];
ROI_beta_list = [];
% Loading sample data
vmp = xff('CG_Objects_SCCAI_3DMCT_SD3DSS4.00mm_THPGLMF2c_TAL_Trials.vmp');

for j = 1:6  ## Found an error running this in OS mac, worked in Windows
    ROI_beta_list =[ROI_beta_list vmp.Map(j).VMPData(meta.indicesIn3D)];
end

subj = init_object(subj,'pattern','2GammaBetaZ');
subj = set_mat(subj,'pattern','2GammaBetaZ',double(ROI_beta_list));
subj.patterns{1}.masked_by = mask_name;
% Loading Conditions and runs
conds = repmat([1 1 1 0 0 0; 0 0 0 1 1 1],1,1);
subj = init_object(subj,'regressors','conds');
subj = set_mat(subj,'regressors','conds', conds);
condnames = {'intra','inter'};
subj = set_objfield(subj,'regressors','conds','condnames',condnames);
subj = init_object(subj,'selector','runs');
runs = [1 2 3 4 5 6];
subj = set_mat(subj,'selector','runs',runs);
subj = create_xvalid_indices(subj,'runs');
% Running classification, training and test
class_args.train_funct_name = 'train_libsvm';
class_args.test_funct_name = 'test_libsvm';
save('CG_OBJECTS_ROI_MVPA','results');

## MVPA using PyMVPA <a id="pymvpa"></a>

As long as this uses Python code we decided to create an independent notebook related to it